### 주제

---

🧪 **금융 고객데이터에 대한 클러스터링 시행 및 군집별 인사이트 도출**

- 데이터셋에 대한 현황을 파악할 수 있도록 합니다.
- 다양한 condition(군집갯수, 컬럼갯수)을 통해 결과를 비교하도록 합니다.
- 군집(클러스터)별로 유의미한 특징을 살펴보고 이를 시각화 하겠습니다.
- 이를 바탕으로 군집별 인사이트를 제시할 수 있도록 합니다.

In [113]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.

sns.set_palette('Set3')

---

## 파생변수

In [ ]:
card = pd.read_csv('./data//new_card_data.csv')
trans = pd.read_csv('./data/new_transactions_data.csv')
user = pd.read_csv('./data/new_users_data.csv')

user = user.rename(columns={'id': 'client_id'})
card = card.rename(columns={'id': 'card_id'})
trans = trans.rename(columns={'id': 'trans_id'})

In [139]:
card_type_df = card[['card_id','card_type']]
trans = trans.merge(card_type_df, on = 'card_id', how = 'inner')
trans.head(5)

,trans_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,year,month,weekday,card_type
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No,2010,1,4,Debit (Prepaid)
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No,2010,1,4,Credit
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No,2010,1,4,Debit
3,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,No,2010,1,4,Debit
4,7475333,2010-01-01 00:07:00,1807,165,4.81,Swipe Transaction,20519,Bronx,NY,10464.0,5942,No,2010,1,4,Debit (Prepaid)


### 1. User
- 은퇴 나이 - 현재 나이
- 부채/연소득

In [140]:
user['years_to_retirement'] = user['retirement_age']-user['current_age']

In [141]:
user['debt_to_annual_income_ratio'] = (user['total_debt']/user['yearly_income']).round(2)

### 2. Trans (고객별)
1. 환불 유무
2. 잔액 부족으로 인한 거래 에러 비율
3. 고객별 카드별 연평균 사용량
4. 고객별 카드 별 환불 비율


In [142]:
# 환불
trans['refund'] = trans['amount'].apply(lambda x: True if x <0 else False)

In [143]:
# 잔액 부족으로 인한 에러 비율
count_trans = trans.groupby(['client_id']).size().to_frame('trans_count')
error_trans = trans[trans['errors']=='Insufficient Balance'].groupby(['client_id']).size().to_frame('error_trans_count')
error_ratio = (error_trans['error_trans_count']/count_trans['trans_count']).round(4)

user['error_ratio'] = user["client_id"].map(error_ratio)

In [144]:
# 고객별 카드 별 연 평균 사용 횟수

# 고객별 연도별 총 카드 사용 횟수
client_yearly_total_count = trans.groupby(['client_id','card_type','year']).size().reset_index(name='count')
# 고객별 연평균 카드 사용 횟수
client_avg_yearly_count = client_yearly_total_count.groupby(['client_id','card_type'])['count'].mean().round(0).reset_index(name='avg_year_count')

card = card.merge(client_avg_yearly_count, on= ['card_type','client_id'], how='left')

In [145]:
# 고객별 카드 별 환불 비율

# 고객별 연도별 총 카드 사용 횟수
client_total_count = trans.groupby(['client_id','card_type']).size().reset_index(name='count')
# 고객별 카드별 환불
client_refund_count = trans[trans['amount']<0].groupby(['client_id','card_type']).size().reset_index(name='refund_count')

client_refund = client_total_count.merge(client_refund_count, on=['client_id','card_type'], how='inner')
client_refund['refund_ratio'] = (client_refund['refund_count']/client_refund['count']).round(2)

card = card.merge(client_refund, on= ['card_type','client_id'], how='left')
card.drop(columns=['count','refund_count'],inplace=True)
card.head(5)

,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,acct_open_year,avg_year_count,refund_ratio
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,24295,2002-09-01,2008,No,2002,597.0,0.03
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,21968,2014-04-01,2014,No,2014,597.0,0.03
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,46414,2003-07-01,2004,No,2003,597.0,0.03
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,12400,2003-01-01,2012,No,2003,541.0,0.03
4,4537,1746,Visa,Credit,4404898874682993,2003-09-01,736,YES,1,27500,2003-09-01,2012,No,2003,NaN,NaN


### 3. Card
- 가입기간: 2019.10.31 - acct_open_date
- 고객별 카드 브랜드 갯수

In [146]:
print(max(card['acct_open_date']))
current_date = pd.to_datetime('2020-02')
acc_year = pd.to_datetime(card['acct_open_date']).dt.year
acc_month = pd.to_datetime(card['acct_open_date']).dt.month
card['subscription_period(month)'] = (current_date.year-acc_year) * 12 + (current_date.month-acc_month)

2020-02-01


In [147]:
brand_group = card.groupby(['client_id'])['card_brand'].nunique()
user['use_card_brand'] = user["client_id"].map(brand_group)

### User + Trans
- 고객별 연평균 소비 / 고객별 연소득

In [ ]:
# 고객별 연도별 총 소비금액
client_yearly_total_spend = trans.groupby(['client_id','year'])['amount'].sum().reset_index()
# 고객별 연평균 소비금액
client_avg_yearly_spend = client_yearly_total_spend.groupby(['client_id'])['amount'].mean().reset_index(name='avg_year_amount')
client_avg_yearly_spend
user = user.merge(client_avg_yearly_spend, left_on='client_id',right_on= 'client_id', how='left')

user['card_spending_to_annual_income_ratio'] = (user['avg_year_amount']/user['yearly_income']).round(4)
user.drop(columns='avg_year_amount',inplace=True)

### 결측치 확인

In [151]:
user.isna().sum()

client_id                                 0
current_age                               0
retirement_age                            0
birth_year                                0
birth_month                               0
gender                                    0
address                                   0
latitude                                  0
longitude                                 0
per_capita_income                         0
yearly_income                             0
total_debt                                0
credit_score                              0
num_credit_cards                          0
log_yearly_income                         0
log_per_capita_income                     0
years_to_retirement                       0
debt_to_annual_income_ratio               0
error_ratio                             790
use_card_brand                           43
card_spending_to_annual_income_ratio    789
dtype: int64

In [ ]:
#  보유 카드 수가 있는데 사용하는 카드 브랜드 수가 없다? → 이상치 → drop
user.drop(index= list(user[(user['use_card_brand'].isna())].index),inplace=True)

,client_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,log_yearly_income,log_per_capita_income,years_to_retirement,debt_to_annual_income_ratio,error_ratio,use_card_brand,card_spending_to_annual_income_ratio
43,1231,26,52,1994,1,Female,613 Little Creek Lane,26.41,-81.42,10819,22066,38967,842,1,10.001839,9.289152,26,1.77,NaN,NaN,NaN
69,633,36,69,1983,10,Female,5506 Fifth Boulevard,33.88,-118.27,24611,50179,110515,743,1,10.823372,10.110989,33,2.20,NaN,NaN,NaN
188,1188,24,61,1995,8,Female,66992 Second Street,36.30,-115.24,31374,63969,128412,626,1,11.066170,10.353767,37,2.01,NaN,NaN,NaN
190,1479,32,60,1987,11,Male,335 Main Street,33.72,-84.42,13059,26631,33950,805,1,10.189869,9.477309,28,1.27,NaN,NaN,NaN
217,1273,21,71,1998,4,Male,966 Sussex Boulevard,41.88,-87.97,24223,49382,43126,762,1,10.807362,10.095099,50,0.87,NaN,NaN,NaN
309,1172,50,64,1970,2,Male,5154 Sixth Avenue,38.02,-121.30,14709,29994,100834,769,1,10.308786,9.596283,14,3.36,NaN,NaN,NaN
320,768,53,67,1966,10,Female,709 River Lane,40.13,-75.06,34043,69412,177225,620,1,11.147829,10.435409,14,2.55,NaN,NaN,NaN
324,1610,42,66,1978,1,Female,12150 Mill Boulevard,32.32,-90.20,13717,27967,50159,690,2,10.238816,9.526464,24,1.79,NaN,NaN,NaN
326,1308,36,67,1984,2,Female,28 Lexington Drive,47.36,-122.10,28919,58964,71890,619,1,10.984699,10.272289,31,1.22,NaN,NaN,NaN
327,780,74,66,1945,11,Male,41 Rose Lane,42.35,-71.22,56720,58378,73124,652,1,10.974712,10.945900,-8,1.25,NaN,NaN,NaN


In [ ]:
user[(user['error_ratio'].isna()) & (user['card_spending_to_annual_income_ratio'].isna())][:5]
# → 거래 기록이 없다.

,client_id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,log_yearly_income,log_per_capita_income,years_to_retirement,debt_to_annual_income_ratio,error_ratio,use_card_brand,card_spending_to_annual_income_ratio
7,1711,26,67,1993,12,Male,1941 Ninth Street,45.51,-122.64,26790,54623,114711,728,1,10.908229,10.195821,41,2.10,NaN,1.0,NaN
10,192,27,66,1992,6,Male,888 Fifth Lane,38.65,-121.25,27548,56164,15224,761,2,10.936049,10.223722,39,0.27,NaN,1.0,NaN
11,640,29,63,1990,9,Female,8677 Littlewood Lane,40.42,-104.74,22427,45727,94016,629,1,10.730466,10.018065,34,2.06,NaN,1.0,NaN
12,1679,18,67,2002,1,Female,829 Fourth Boulevard,41.76,-71.48,33914,69149,89214,776,1,11.144033,10.431613,49,1.29,NaN,1.0,NaN
14,1590,48,62,1971,5,Female,781 East Street,29.45,-98.50,10059,20513,32509,599,1,9.928863,9.216322,14,1.58,NaN,1.0,NaN


In [132]:
card.isna().sum()

card_id                          0
client_id                        0
card_brand                       0
card_type                        0
card_number                      0
expires                          0
cvv                              0
has_chip                         0
num_cards_issued                 0
credit_limit                     0
acct_open_date                   0
year_pin_last_changed            0
card_on_dark_web                 0
acct_open_year                   0
avg_year_count                1599
refund_ratio                  1622
subscription_period(month)       0
dtype: int64

In [ ]:
card[card.isna( ).any(axis=1)]
# →  거래 기록이 없다

,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,acct_open_year,avg_year_count,refund_ratio,subscription_period(month)
4,4537,1746,Visa,Credit,4404898874682993,2003-09-01,736,YES,1,27500,2003-09-01,2012,No,2003,NaN,NaN,197
22,744,1711,Visa,Debit,4595831603224644,2023-05-01,689,NO,2,17480,2020-01-01,2020,No,2020,NaN,NaN,1
29,1766,192,Visa,Debit,4857799018106614,2021-07-01,930,YES,1,25499,2020-02-01,2020,No,2020,NaN,NaN,0
30,1767,192,Visa,Debit,4324062360619923,2020-02-01,358,YES,1,18641,2020-02-01,2020,No,2020,NaN,NaN,0
31,745,640,Visa,Debit,4852639114720724,2023-05-01,995,YES,1,26019,2020-01-01,2020,No,2020,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5514,957,588,Visa,Debit,4331126149037342,2021-02-01,920,YES,2,21787,2020-01-01,2020,No,2020,NaN,NaN,1
5515,439,588,Discover,Credit,6209823400357197,2020-01-01,916,YES,2,12000,2020-01-01,2020,No,2020,NaN,NaN,1
5526,1305,1007,Mastercard,Credit,5929512204765914,2020-08-01,237,NO,2,9200,2012-02-01,2012,No,2012,NaN,NaN,96
5527,743,1110,Mastercard,Debit,5589768928167462,2020-01-01,630,YES,1,28074,2020-01-01,2020,No,2020,NaN,NaN,1


## user+card만 merge
- 이때 Trans에서 온 파생변수가 Nan인 값이 존재 -> 해당 파생변수들이 모두 Nan이면 고객과 카드 데이터는 있지만, 거래한 내역이 없는거니 fillna(0)  
                                                -> 나머지는 이상치로 dropna

In [175]:
user_col = ['client_id', 'gender', 'credit_score', 'num_credit_cards', 
            'years_to_retirement', 'debt_to_annual_income_ratio', 'error_ratio', 'use_card_brand', 'card_spending_to_annual_income_ratio']
card_col = ['card_id', 'client_id', 'card_type', 'credit_limit', 'avg_year_count', 'refund_ratio', 'subscription_period(month)']

In [176]:
final_user = user[user_col].copy()
final_card = card[card_col].copy()

In [177]:
merge = final_user.merge(final_card, on='client_id', how='left')
print(merge.shape)
merge.head()

(5405, 15)


,client_id,gender,credit_score,num_credit_cards,years_to_retirement,debt_to_annual_income_ratio,error_ratio,use_card_brand,card_spending_to_annual_income_ratio,card_id,card_type,credit_limit,avg_year_count,refund_ratio,subscription_period(month)
0,825,Female,787,5,13,2.14,0.0227,1.0,1.5284,4524,Debit,24295,597.0,0.03,209
1,825,Female,787,5,13,2.14,0.0227,1.0,1.5284,2731,Debit,21968,597.0,0.03,70
2,825,Female,787,5,13,2.14,0.0227,1.0,1.5284,3701,Debit,46414,597.0,0.03,199
3,825,Female,787,5,13,2.14,0.0227,1.0,1.5284,42,Credit,12400,541.0,0.03,205
4,1746,Female,701,5,15,2.48,0.0111,2.0,0.2967,4537,Credit,27500,NaN,NaN,197


In [178]:
merge.isna().sum()

client_id                                  0
gender                                     0
credit_score                               0
num_credit_cards                           0
years_to_retirement                        0
debt_to_annual_income_ratio                0
error_ratio                             1465
use_card_brand                             0
card_spending_to_annual_income_ratio    1464
card_id                                    0
card_type                                  0
credit_limit                               0
avg_year_count                          1576
refund_ratio                            1593
subscription_period(month)                 0
dtype: int64

In [179]:
merge[merge.isna( ).any(axis=1)]


,client_id,gender,credit_score,num_credit_cards,years_to_retirement,debt_to_annual_income_ratio,error_ratio,use_card_brand,card_spending_to_annual_income_ratio,card_id,card_type,credit_limit,avg_year_count,refund_ratio,subscription_period(month)
4,1746,Female,701,5,15,2.48,0.0111,2.0,0.2967,4537,Credit,27500,NaN,NaN,197
22,1711,Male,728,1,41,2.10,NaN,1.0,NaN,744,Debit,17480,NaN,NaN,1
29,192,Male,761,2,39,0.27,NaN,1.0,NaN,1766,Debit,25499,NaN,NaN,0
30,192,Male,761,2,39,0.27,NaN,1.0,NaN,1767,Debit,18641,NaN,NaN,0
31,640,Female,629,1,34,2.06,NaN,1.0,NaN,745,Debit,26019,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5390,588,Male,844,3,45,1.61,NaN,2.0,NaN,957,Debit,21787,NaN,NaN,1
5391,588,Male,844,3,45,1.61,NaN,2.0,NaN,439,Credit,12000,NaN,NaN,1
5402,1007,Male,618,1,-6,0.50,NaN,1.0,NaN,1305,Credit,9200,NaN,NaN,96
5403,1110,Female,673,2,39,2.75,NaN,2.0,NaN,743,Debit,28074,NaN,NaN,1


In [180]:
no_use = merge[(merge['error_ratio'].isna()) & (merge['card_spending_to_annual_income_ratio'].isna())&(merge['avg_year_count'].isna()) & (merge['refund_ratio'].isna())].index
cols_to_fill = ['error_ratio', 'card_spending_to_annual_income_ratio', 'avg_year_count', 'refund_ratio']
merge.loc[no_use, cols_to_fill] = merge.loc[no_use, cols_to_fill].fillna(0)
merge.dropna(axis=0, inplace=True)

In [183]:
print(merge.shape)
merge.isna().sum()

(5275, 15)


client_id                               0
gender                                  0
credit_score                            0
num_credit_cards                        0
years_to_retirement                     0
debt_to_annual_income_ratio             0
error_ratio                             0
use_card_brand                          0
card_spending_to_annual_income_ratio    0
card_id                                 0
card_type                               0
credit_limit                            0
avg_year_count                          0
refund_ratio                            0
subscription_period(month)              0
dtype: int64

In [182]:
merge.to_csv('./data/credit_merge.csv', index=False)